In [3]:
import pandas as pd
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

In [4]:
df = pd.read_csv('Employment_contract_synthetic_w_updated_ragas_v2.csv')
df

,Clause,Truth_Value,Responses,actual_response,actual_truth_value
0,EMPLOYMENT AGREEMENT\nThis Employment Agreemen...,True,"Yes, the clause identifies the involved partie...",Yes : This clause appears to conform with gene...,True
1,"NOW, THEREFORE, in consideration of the mutual...",True,"Yes, the clause outlines definitions and inter...",Yes : This clause appears to conform with gene...,True
2,guidelines.\n2.2 Duties:\n a. The Employee sh...,True,"Yes, the clause outlines the employee's duties...",Yes : The termination clause permits terminati...,True
3,4. Compensation and Benefits\n4.1 Salary: The ...,False,"No, the clause does not fully adhere to standa...",No : The compensation clause offers a salary o...,False
4,extra compensation.\n5.2 Leave: The Employee i...,False,"No, the clause does not fully adhere to standa...",No : This clause imposes overly broad restrict...,False
5,6.2 Non-Compete:\n a. The Employee agrees tha...,False,"No, the clause does not fully adhere to standa...",No : This clause imposes overly broad restrict...,False
6,"b. In some cases, the Employer may require a n...",False,"No, the clause does not fully adhere to standa...",No : The termination clause permits terminatio...,False
7,7.4 Ambiguity: The grounds for termination are...,False,"No, the clause does not fully adhere to standa...",No : The termination clause permits terminatio...,False
8,those laws.\n9. Miscellaneous Provisions\n9.1 ...,False,"Answer: No, the clause does not fully adhere t...",No : This clause appears to conform with gener...,False
9,unequal treatment.\n10. Execution\nIN WITNESS ...,False,"Answer: No, the clause does not fully adhere t...",No : This clause appears to conform with gener...,False


In [5]:
PROJECT_DESCRIPTION = (
    "This project automates business contract validation by ensuring that contracts meet predefined standards. "
    "The system classifies input contracts and compares them against standard templates (including key provisions "
    "from the Indian Contract Law and the Indian Contract Act) to detect any deviations. "
    "Each clause must be evaluated for compliance with these legal standards. "
)

EXAMPLE_EVALUATION = (
    "For example, consider the clause: 'The contractor shall maintain confidentiality for 3 years after termination.' "
    "If the standard requires a confidentiality period of at least 2 years, then the correct evaluation would be 'Yes', "
    "with a rationale such as: 'The clause meets the minimum confidentiality period required by the standards.'"
)

CHAT_PROMPT_TEMPLATE = ChatPromptTemplate(
    [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content="Project Description: " + PROJECT_DESCRIPTION
        ),
        ChatMessage(
            role=MessageRole.USER,
            content="Example Evaluation: " + EXAMPLE_EVALUATION
        ),
        ChatMessage(
            role= MessageRole.SYSTEM,
            content= "History (for context only): {history}"
        ),
        ChatMessage(
            role=MessageRole.USER,
            content=(
                "Now, please evaluate whether the following contract clause adheres to the above standards. "
                "Respond with 'Yes' or 'No' and provide a brief rationale.\nClause: {clause}"
            )
        )
    ]
)
# query_prompt = CHAT_PROMPT_TEMPLATE.format(history=history,clause=clause)

In [7]:
query= []
for idx,i in enumerate(df['Clause']):
    if idx ==0:
        history = ""
    else:
        history = f"Previous Clause:{df.loc[idx-1,'Clause']}\nPrevious Response: {df.loc[idx-1,'Responses']}"
    query.append(CHAT_PROMPT_TEMPLATE.format(history=history,clause=i))

In [9]:
query[1]

"system: Project Description: This project automates business contract validation by ensuring that contracts meet predefined standards. The system classifies input contracts and compares them against standard templates (including key provisions from the Indian Contract Law and the Indian Contract Act) to detect any deviations. Each clause must be evaluated for compliance with these legal standards. \nuser: Example Evaluation: For example, consider the clause: 'The contractor shall maintain confidentiality for 3 years after termination.' If the standard requires a confidentiality period of at least 2 years, then the correct evaluation would be 'Yes', with a rationale such as: 'The clause meets the minimum confidentiality period required by the standards.'\nsystem: History (for context only): Previous Clause:EMPLOYMENT AGREEMENT\nThis Employment Agreement (the “Agreement”) is made and entered into on this 5th day of April, 2025 by and between:\nAcme Innovations Pvt. Ltd.\na private limit

In [10]:
df['Query'] = query

In [11]:
df

,Clause,Truth_Value,Responses,actual_response,actual_truth_value,Query
0,EMPLOYMENT AGREEMENT\nThis Employment Agreemen...,True,"Yes, the clause identifies the involved partie...",Yes : This clause appears to conform with gene...,True,system: Project Description: This project auto...
1,"NOW, THEREFORE, in consideration of the mutual...",True,"Yes, the clause outlines definitions and inter...",Yes : This clause appears to conform with gene...,True,system: Project Description: This project auto...
2,guidelines.\n2.2 Duties:\n a. The Employee sh...,True,"Yes, the clause outlines the employee's duties...",Yes : The termination clause permits terminati...,True,system: Project Description: This project auto...
3,4. Compensation and Benefits\n4.1 Salary: The ...,False,"No, the clause does not fully adhere to standa...",No : The compensation clause offers a salary o...,False,system: Project Description: This project auto...
4,extra compensation.\n5.2 Leave: The Employee i...,False,"No, the clause does not fully adhere to standa...",No : This clause imposes overly broad restrict...,False,system: Project Description: This project auto...
5,6.2 Non-Compete:\n a. The Employee agrees tha...,False,"No, the clause does not fully adhere to standa...",No : This clause imposes overly broad restrict...,False,system: Project Description: This project auto...
6,"b. In some cases, the Employer may require a n...",False,"No, the clause does not fully adhere to standa...",No : The termination clause permits terminatio...,False,system: Project Description: This project auto...
7,7.4 Ambiguity: The grounds for termination are...,False,"No, the clause does not fully adhere to standa...",No : The termination clause permits terminatio...,False,system: Project Description: This project auto...
8,those laws.\n9. Miscellaneous Provisions\n9.1 ...,False,"Answer: No, the clause does not fully adhere t...",No : This clause appears to conform with gener...,False,system: Project Description: This project auto...
9,unequal treatment.\n10. Execution\nIN WITNESS ...,False,"Answer: No, the clause does not fully adhere t...",No : This clause appears to conform with gener...,False,system: Project Description: This project auto...


In [12]:
df.to_csv('Final_Ragas_Evaluation_dataset_with_RAG_Responses.csv',index=False)